In [117]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

world_cup_matches = pd.read_csv("C:\\Users\\ianfr\\Documents\\dev\\inteligencia-computacional\\dataset\\results.csv")
world_cup_matches['year'] = world_cup_matches['date'].str[:4].astype(int)
world_cup_matches['month'] = world_cup_matches['date'].str[5:7].astype(int)
world_cup_matches['day'] = world_cup_matches['date'].str[8:].astype(int)
world_cup_matches = world_cup_matches[['home_team', 'home_score', 'away_score', 'away_team', 'year', 'month', 'day']]
world_cup_matches = world_cup_matches[(world_cup_matches.year) >= 1930]
world_cup_matches = world_cup_matches[(world_cup_matches.year) <= 2018]
teams = ['Russia', 'Brazil', 'Australia', 'Iran', 'Japan', 'Saudi Arabia', 'South Korea', 'Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Colombia', 'Peru', 'Uruguay', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Serbia', 'Spain', 'Sweden', 'Switzerland']
teams.sort()
world_cup_matches = world_cup_matches[world_cup_matches.home_team.isin(teams) | world_cup_matches.away_team.isin(teams)]
world_cup_matches.head()

,home_team,home_score,away_score,away_team,year,month,day
1252,Spain,1,0,Czechoslovakia,1930,1,1
1253,Portugal,1,0,Czechoslovakia,1930,1,12
1256,Italy,4,2,Switzerland,1930,2,9
1259,Portugal,2,0,France,1930,2,23
1260,Germany,0,2,Italy,1930,3,2


In [118]:
world_cup_matches.describe()

,home_score,away_score,year,month,day
count,16594.000000,16594.000000,16594.000000,16594.000000,16594.000000
mean,1.688743,1.160118,1988.540918,6.590936,15.381885
std,1.656105,1.306737,21.683312,3.159086,8.537780
min,0.000000,0.000000,1930.000000,1.000000,1.000000
25%,1.000000,0.000000,1974.000000,4.000000,8.000000
50%,1.000000,1.000000,1993.000000,6.000000,15.000000
75%,2.000000,2.000000,2006.000000,9.000000,23.000000
max,31.000000,17.000000,2018.000000,12.000000,31.000000


In [172]:
data = {'Team': teams, 'GoalsScoredMean': [0 for i in range(32)], 'GoalsConcededMean': [0 for i in range(32)]}
stats = pd.DataFrame(data)
teams_att_power = []
teams_def_power = []
index = 0
for time in teams:
    team_goals = 0.0
    conceded_goals = 0.0
    team_matches = 0.0
    time_model = world_cup_matches[(world_cup_matches['home_team'] == time) | (world_cup_matches['away_team'] == time)]
    #print(time_model)
    for index, row in time_model.iterrows():
        if row['home_team'] == time:
            team_goals += row['home_score']
            conceded_goals += row['away_score']
        if row['away_team'] == time:
            team_goals += row['away_score']
            conceded_goals += row['home_score']
        team_matches += 1
    teams_att_power.append(team_goals/(team_matches))
    teams_def_power.append(conceded_goals/team_matches)
    index += 1
    stats.loc[stats['Team'] == time, 'GoalsScoredMean'] = team_goals/(team_matches)
    stats.loc[stats['Team'] == time, 'GoalsConcededMean'] = conceded_goals/(team_matches)
    
stats2 = stats.set_index('Team')
#print(stats2)
world_cup_matches['HomeAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['HomeDefence'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayDefence'] = [0.0 for i in range(len(world_cup_matches))]

for index, row in world_cup_matches.iterrows():
    if (row['home_team'] in teams and row['away_team'] in teams):
        world_cup_matches.loc[index,'HomeAttack'] = stats2.loc[row['home_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'HomeDefence'] = stats2.loc[row['home_team'], 'GoalsConcededMean']
        world_cup_matches.loc[index,'AwayAttack'] = stats2.loc[row['away_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'AwayDefence'] = stats2.loc[row['away_team'], 'GoalsConcededMean']
    elif (row['home_team'] in teams and row['away_team'] not in teams):
        world_cup_matches.loc[index,'HomeAttack'] = stats2.loc[row['home_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'HomeDefence'] = stats2.loc[row['home_team'], 'GoalsConcededMean']
    else:
        world_cup_matches.loc[index,'AwayAttack'] = stats2.loc[row['away_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'AwayDefence'] = stats2.loc[row['away_team'], 'GoalsConcededMean']
world_cup_matches.head()

,home_team,home_score,away_score,away_team,year,month,day,HomeAttack,HomeDefence,AwayAttack,AwayDefence
1252,Spain,1,0,Czechoslovakia,1930,1,1,1.978852,0.913897,0.000000,0.000000
1253,Portugal,1,0,Czechoslovakia,1930,1,12,1.675170,1.137755,0.000000,0.000000
1256,Italy,4,2,Switzerland,1930,2,9,0.000000,0.000000,1.440510,1.586402
1259,Portugal,2,0,France,1930,2,23,1.675170,1.137755,1.779255,1.135638
1260,Germany,0,2,Italy,1930,3,2,2.238259,1.079038,0.000000,0.000000


In [129]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([world_cup_matches[['home_team','away_team', 'HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence', 'home_score']].rename(
            columns={'home_team':'team', 'away_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def','home_score':'goals'}),
           world_cup_matches[['away_team','home_team','HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence','away_score']].rename(
            columns={'away_team':'team', 'home_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def','away_score':'goals'})])
#print(goal_model_data)
poisson_model = smf.glm(formula="goals ~ att + defe + away_att + away_def", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                33188
Model:                            GLM   Df Residuals:                    33183
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -54856.
Date:                Fri, 04 Oct 2019   Deviance:                       52468.
Time:                        01:22:57   Pearson chi2:                 5.28e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3477      0.014     25.582      0.000       0.321       0.374
att            0.0944      0.012      7.978      0.000       0.071       0.118
defe          -0.0895      0.018     -4.955      0.000      -0.125      -0.054
away_att      -0.0497      0.012     -4.007      0.000      -0.074      -0.025
away_def       0.0232      0.018      1.280      0.201      -0.012       0.059
==============================================================================
"""

In [190]:
def get_team_stats(team, return_df=False):
    #team_stats_df = player_overall_df[mask].head(no_of_players).groupby(groupby_cols).mean().round(0)
    #if return_df == True:
    #    return team_stats_df
    #else:
    #df = stats2[team]
    attack = stats2.loc[team, 'GoalsScoredMean']
    defence = stats2.loc[team, 'GoalsConcededMean']
    return [attack, defence]

def simulate_goals(game, a_team='A Team', b_team='B Team', verbose=False):
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals


def show_game_summary(game, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(game, a_team, b_team, verbose=verbose)
    
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals


def simulate_match(foot_model, a_team, b_team, max_goals=5, verbose=False):
    a_team_stats = get_team_stats(a_team)
    b_team_stats = get_team_stats(b_team)
    
    data1 = {'att': a_team_stats[0],
              'defe': a_team_stats[1],
              'away_att': b_team_stats[0], 
              'away_def': b_team_stats[1]}
    
    data2 = {'att': b_team_stats[0],
              'defe': b_team_stats[1],
              'away_att': a_team_stats[0], 
              'away_def': a_team_stats[1]}
        
    a_goals_avg = foot_model.predict(pd.DataFrame(data=data1, index=[1])).values[0]
    b_goals_avg = foot_model.predict(pd.DataFrame(data=data2, index=[1])).values[0]
    print(a_goals_avg)
    print(b_goals_avg)
    
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [a_goals_avg, b_goals_avg]]
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, a_team, b_team, verbose=verbose)
    return game

In [195]:
game = simulate_match(poisson_model, 'Brazil', 'Colombia')
show_game_summary(game, 'Brazil', 'Colombia', verbose=True)

1.5579302494757379
1.3069521331141798
Brazil chance of Win: 42.72%
Colombia chance of Win: 31.79%
Odds of a draw: 24.73%
Random float: 0.6779737273205324
Brazil goals: 2
Colombia goals: 1



(0.42722256067721953, 0.31786937351386024, 0.24729975082418648, 2, 1)

In [196]:
pd.DataFrame(game)

,0,1,2,3,4,5
0,0.056990,0.074483,0.048673,0.021204,0.006928,0.001811
1,0.088786,0.116039,0.075829,0.033035,0.010794,0.002821
2,0.069161,0.090391,0.059068,0.025733,0.008408,0.002198
3,0.035916,0.046941,0.030675,0.013363,0.004366,0.001141
4,0.013989,0.018283,0.011947,0.005205,0.001701,0.000445
5,0.004359,0.005697,0.003723,0.001622,0.000530,0.000139
